# import

In [1]:
import numpy as np
import pandas as pd
import glob
import sqlite3
import datetime

# funcs

In [2]:
def load_csv(load_path):
    df = pd.read_csv(load_path, index_col=0)
    return df

# classes

In [3]:
class DataFramePreProcessing():

    
    def __init__(self, path_, is_daw=False):
        self.path_ = path_
        self.is_daw = is_daw

        
    def load_df(self):
        if self.is_daw:
            d='d'
        else:
            d=''
        FILE = glob.glob(self.path_)
        df = pd.read_csv(FILE[0])
        df = df.rename(columns={df.columns[0]:'nan',df.columns[1]:'nan',df.columns[2]:'nan',\
                                    df.columns[3]:'day',df.columns[4]:'nan',df.columns[5]:d+'open',\
                                    df.columns[6]:d+'high',df.columns[7]:d+'low',df.columns[8]:d+'close',\
                                       df.columns[9]:d+'volume',})
        df = df.drop('nan',axis=1)
        df = df.drop(df.index[0])
        df['day'] = pd.to_datetime(df['day'],format='%Y/%m/%d')
        df.set_index('day',inplace=True)

        return df.astype(float)
    
class DBInterface():
    
    
    def __init__(self):
        self.db = None
        self.is_initialized = False
        
        
    def make_db(self,path_):
        if not self.is_initialized:
            db = sqlite3.connect(
                    path_,              #ファイル名
                    isolation_level=None,
                    )
            self.db = db
        else:
            print("already initialized")
        
    
    def load_db(self, path_):
        if not self.is_initialized:
            db = sqlite3.connect(
                    path_,              #ファイル名
                    isolation_level=None,
                    )
            self.db = db
        else:
            print("already initialized")
        
        
    def show_tables(self):
        sql = """SELECT NAME FROM sqlite_master WHERE TYPE='table'; """
        for i in self.db.execute(sql):
            print(i[0])
            
            
    def create_table(self,sql):
        self.db.execute(sql)
        
        
    def drop_table(self, table_name):
        self.db.execute("""DROP TABLE IF EXISTS %s""" % table_name)
        
    
    def show_column(self, table):
        self.db.execute("""SELECT * FROM %s""" % table)
        names = list(map(lambda x: x[0], self.db.cursor().description))
        print(names)
        
        
    def select_show(self, attribute, table):
        for item in self.db.execute("""SELECT id FROM Meigara WHERE name = %s""" % table):
            print(item)
            
            
    def return_db(self):
        return self.db
    
    
    def make_Results_tuple(self, df, id_):
        Hiashis = pd.DataFrame()
        Hiashis['id'] = [id_] * len(df)
        Hiashis['day'] = df.reset_index()['day'].astype(str)
        Hiashis['open'] = df['open'].tolist()
        Hiashis['high'] = df['high'].tolist()
        Hiashis['low'] = df['low'].tolist()
        Hiashis['close'] = df['close'].tolist()
        Hiashis['volume'] = df['volume'].tolist()
        
        return list(Hiashis.itertuples(index=False, name=None)) 
    
    
    def execute_many_Hiashis(self, df, name):
        id_ = self.return_id(name)
        try:
            Hiashis = self.make_Hiashis_tuple(df,id_)
            self.db.executemany( "INSERT INTO Hiashis VALUES (?,?,?,?,?,?,?)", Hiashis )
        except :
            print("UNIQUE constraint failed: Hiashis.id, Hiashis.date")

        
    def return_id(self, name):
        sql = """SELECT id FROM Meigaras WHERE name='%s' """ % name
        c = self.db.cursor()
        id_ = c.execute(sql).fetchone()[0]
        return id_
    
    
    def update_Hiashi(self,df,name):
        try:
            id_ = self.return_id(name)
        except:
            print("No such Meigaras.name !")
        
        sql = """ SELECT MAX(Hiashis.date) FROM Hiashis WHERE Hiashis.meigara_id = %s""" %id_
        table_date = self.db.execute(sql).fetchone()[0] 
        
        try:
            new_df = df[df.index>table_date]
            Hiashis = self.make_Hiashis_tuple(new_df,id_)
            self.db.executemany( "INSERT INTO Hiashis VALUES (?,?,?,?,?,?,?)", Hiashis )
        except:
            print("Insert failed.")
            
    
    
    def make_Ayumines_tuple(self, df,id_):
######### date どないすんねん
        date_ = None
        Ayumines = pd.DataFrame()
        Ayumines['id'] = [id_] * len(df)
        Ayumines['date'] = [date_] * len(df)#df.reset_index()['時刻'].astype(str)
        Ayumines['time'] = df['時刻'].tolist()
        Ayumines['price'] = df['値段'].tolist()
        Ayumines['volume'] = df['株数'].tolist()
        Ayumines['total_price'] = df['金額'].tolist()

        return list(Ayumines.itertuples(index=False, name=None)) 
    
    
    def execute_many_Ayumines(self, df, name):
        id_ = self.return_id(name)
        try:
            Ayumines = self.make_Ayumines_tuple(df,id_)
            self.db.executemany( "INSERT INTO Ayumines VALUES (?,?,?,?,?,?)", Ayumines )
        except :
            print("UNIQUE constraint failed: Hiashis.id, Hiashis.date")
            
        
    def update_Ayumine(self,df,name):
        try:
            id_ = self.return_id(name)
        except:
            print("No such Meigaras.name !")
        
        sql = """ SELECT MAX(Hiashis.date) FROM Hiashis WHERE Hiashis.meigara_id = %s""" %id_
        table_date = self.db.execute(sql).fetchone()[0] 
        
        try:
            new_df = df[df.index>table_date]
            Hiashis = self.make_Hiashis_tuple(new_df,id_)
            self.db.executemany( "INSERT INTO Hiashis VALUES (?,?,?,?,?,?,?)", Hiashis )
        except:
            print("Insert failed.")

# 一旦, クラス使わないで, dbいじる

In [92]:
sql = """
CREATE TABLE Test(
id INTEGER PRIMARY KEY AUTOINCREMENT,
name VARCHAR(40)
);"""

In [98]:
sql = """
DROP TABLE Horse_Results;"""
db.execute(sql)

In [93]:
db.execute(sql)

In [96]:
sql = """
INSERT  INTO Test(name) VALUES('hoge_hoge');
"""

In [97]:
db.execute(sql)

In [4]:
db = sqlite3.connect(
    "horse.db",              #ファイル名
    isolation_level=None,
)

In [28]:
sql = """SELECT NAME FROM sqlite_master WHERE TYPE='table'; """
for i in db.execute(sql):
    print(i[0])
    

Results
Horse_Results


In [63]:
horse_results.index.tolist()

[2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017105800,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017106540,
 2017102604,
 2017102604,
 2017102604,
 2017102604,
 2017102604,
 2017102604,
 2017102604,
 2017102604,
 2017102604,
 2017102604,
 2017102604,
 2017102604,
 2017102604,
 2017102604,
 2017102604,
 2017102604,
 2017102604,
 2017102604,
 2017102604,

In [7]:
horse_results.columns

Index(['日付', '開催', '天気', 'R', 'レース名', '映像', '頭数', '枠番', '馬番', 'オッズ', '人気',
       '着順', '騎手', '斤量', '距離', '馬場', '馬場指数', 'タイム', '着差', 'ﾀｲﾑ指数', '通過', 'ペース',
       '上り', '馬体重', '厩舎ｺﾒﾝﾄ', '備考', '勝ち馬(2着馬)', '賞金'],
      dtype='object')

In [29]:
sql = """SELECT * FROM Horse_Results;"""
db.execute(sql)

In [99]:
sql = """CREATE TABLE Horse_Results(
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        horse_id INTEGER,
        date VARCHAR(40),
        kaisai VARCHAR(40),
        tenki VARCHAR(40),
        round INTEGER,
        race_name VARCHAR(40),
        tousu INTEGER,
        wakuban INTEGER,
        umaban INTEGER,
        odds REAL,
        ninki REAL,
        chakujun VARCHAR(40),
        kishu VARCHAR(40),
        kinryo  REAL,
        kyori  VARCHAR(40),
        baba  VARCHAR(20),
        time VARCHAR(20),
        chakusa REAL,
        tsu_ka VARCHAR(20),
        pace VARCHAR(20),
        nobori REAL,
        taiju VARCHAR(20),
        kachiuma VARCHAR(40),
        shokin REAL
    );
"""


# INSERT UMA DB

In [100]:
db.execute(sql)

In [47]:
sql = """

INSERT  INTO Test (name) VALUES ('hoge_hoge')
;
"""

In [66]:
horse_results['天気']

2017105800    曇
2017105800    曇
2017105800    雨
2017105800    晴
2017105800    雨
             ..
2017105400    晴
2017105400    曇
2017105400    曇
2017105400    晴
2017105400    曇
Name: 天気, Length: 225907, dtype: object

In [67]:
horse_results.columns

Index(['日付', '開催', '天気', 'R', 'レース名', '映像', '頭数', '枠番', '馬番', 'オッズ', '人気',
       '着順', '騎手', '斤量', '距離', '馬場', '馬場指数', 'タイム', '着差', 'ﾀｲﾑ指数', '通過', 'ペース',
       '上り', '馬体重', '厩舎ｺﾒﾝﾄ', '備考', '勝ち馬(2着馬)', '賞金'],
      dtype='object')

In [101]:

Horse_Results = pd.DataFrame()
Horse_Results['horse_id'] = horse_results.index.tolist()
Horse_Results['date'] = horse_results['日付'].tolist()#df.reset_index()['時刻'].astype(str)
Horse_Results['kaisai'] = horse_results['開催'].tolist()
Horse_Results['tenki'] = horse_results['天気'].tolist()
Horse_Results['round'] = horse_results['R'].tolist()
Horse_Results['race_name'] = horse_results['レース名'].tolist()
Horse_Results['tousu'] = horse_results['頭数'].tolist()#horse_results.reset_index()['時刻'].astype(str)
Horse_Results['wakuban'] = horse_results['枠番'].tolist()
Horse_Results['umaban'] = horse_results['馬番'].tolist()
Horse_Results['odds'] = horse_results['オッズ'].tolist()
Horse_Results['ninki'] = horse_results['人気'].tolist()
Horse_Results['chakujun'] = horse_results['着順'].tolist()
Horse_Results['kishu'] = horse_results['騎手'].tolist()
Horse_Results['kinryo'] = horse_results['斤量'].tolist()
Horse_Results['kyori'] = horse_results['距離'].tolist()
Horse_Results['baba'] = horse_results['馬場'].tolist()
Horse_Results['time'] = horse_results['タイム'].tolist()
Horse_Results['chakusa'] = horse_results['着差'].tolist()
Horse_Results['tsu_ka'] = horse_results['通過'].tolist()
Horse_Results['pace'] = horse_results['ペース'].tolist()
Horse_Results['nobori'] = horse_results['上り'].tolist()
Horse_Results['taiju'] = horse_results['馬体重'].tolist()
Horse_Results['kachiuma'] = horse_results['勝ち馬(2着馬)'].tolist()
Horse_Results['shokin'] = horse_results['賞金'].tolist()

Horse_Results_tuple =  list(Horse_Results.itertuples(index=False, name=None)) 






In [102]:
Horse_Results_tuple

[(2017105800,
  '2021/07/08',
  '名古屋',
  '曇',
  7.0,
  'C12組',
  10.0,
  4.0,
  4,
  6.6,
  3.0,
  '中',
  '宮下瞳',
  54.0,
  'ダ1400',
  '不',
  nan,
  nan,
  '4-3',
  '0.0-39.5',
  nan,
  '479(-1)',
  'ナツミフルーリー',
  nan),
 (2017105800,
  '2021/06/30',
  '名古屋',
  '曇',
  5.0,
  'C14組',
  10.0,
  3.0,
  3,
  12.2,
  4.0,
  '2',
  '丹羽克輝',
  56.0,
  'ダ1400',
  '良',
  '1:33.3',
  0.7,
  '5-6-5',
  '0.0-40.3',
  39.4,
  '480(-4)',
  'マルカブレーブ',
  10.2),
 (2017105800,
  '2021/06/16',
  '名古屋',
  '雨',
  5.0,
  '美穂ちゃんバースデイ',
  11.0,
  2.0,
  2,
  13.5,
  4.0,
  '5',
  '宮下瞳',
  54.0,
  'ダ1400',
  '重',
  '1:32.5',
  2.7,
  '10-10-8',
  '0.0-38.2',
  38.4,
  '484(+3)',
  'タイキスピネル',
  2.6),
 (2017105800,
  '2021/06/03',
  '名古屋',
  '晴',
  6.0,
  'C14組',
  10.0,
  8.0,
  9,
  3.8,
  2.0,
  '7',
  '尾崎章生',
  56.0,
  'ダ1400',
  '良',
  '1:33.7',
  1.0,
  '3-4-4',
  '0.0-39.6',
  40.1,
  '481(-6)',
  'トーホウボッカ',
  nan),
 (2017105800,
  '2021/05/19',
  '名古屋',
  '雨',
  5.0,
  'C15組',
  11.0,
  8.0,
  11,
  29.8,
  

In [103]:

db.executemany( """
INSERT INTO Horse_Results (horse_id,date,kaisai,tenki,round,race_name,tousu,wakuban,umaban,odds,ninki,chakujun,kishu,kinryo,kyori,baba,time,chakusa,tsu_ka,pace,nobori,taiju,kachiuma,shokin) 
VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)"""
, Horse_Results_tuple)

In [32]:
for data in horse_results:
    print(data)
    break

日付


In [34]:
horse_results

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
2017105800,2021/07/08,名古屋,曇,7.0,C12組,NaN,10.0,4.0,4,6.6,...,NaN,**,4-3,0.0-39.5,NaN,479(-1),NaN,NaN,ナツミフルーリー,NaN
2017105800,2021/06/30,名古屋,曇,5.0,C14組,NaN,10.0,3.0,3,12.2,...,0.7,**,5-6-5,0.0-40.3,39.4,480(-4),NaN,NaN,マルカブレーブ,10.2
2017105800,2021/06/16,名古屋,雨,5.0,美穂ちゃんバースデイ,NaN,11.0,2.0,2,13.5,...,2.7,**,10-10-8,0.0-38.2,38.4,484(+3),NaN,NaN,タイキスピネル,2.6
2017105800,2021/06/03,名古屋,晴,6.0,C14組,NaN,10.0,8.0,9,3.8,...,1.0,**,3-4-4,0.0-39.6,40.1,481(-6),NaN,NaN,トーホウボッカ,NaN
2017105800,2021/05/19,名古屋,雨,5.0,C15組,NaN,11.0,8.0,11,29.8,...,0.5,**,3-3-3,0.0-38.4,38.6,487(0),NaN,NaN,キングリアリティー,10.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017105400,2020/06/20,3阪神5,晴,4.0,3歳未勝利,NaN,18.0,8.0,16,151.2,...,1.1,**,11-10,34.2-36.4,36.6,478(+2),NaN,NaN,トゥールドマジ,NaN
2017105400,2019/11/30,5阪神1,曇,4.0,2歳未勝利,NaN,15.0,4.0,6,47.4,...,1.5,**,12-11,34.0-35.1,35.2,476(+10),NaN,NaN,メイショウチタン,NaN
2017105400,2019/09/28,4阪神8,曇,3.0,2歳未勝利,NaN,7.0,5.0,5,43.7,...,1.4,**,1-1-1-1,37.8-34.6,36.0,466(+2),NaN,NaN,アカイイト,NaN
2017105400,2019/09/07,4中山1,晴,9.0,アスター賞(1勝クラス),NaN,9.0,8.0,9,122.2,...,0.8,**,8-8-7,36.0-33.9,34.0,464(-12),NaN,NaN,サクセッション,NaN


In [27]:
db.execute(sql)

In [79]:
sql = """
DROP TABLE Test;
"""

In [80]:
db.execute(sql)

# path_

# Table

horse_table \
jocky_table \
race_table \
trainer_table \
owner_table \
breeder_table \


In [4]:
save_path = '/Users/rince/Desktop/Horse/Data/saishin/return.csv'
return_tables = load_csv(save_path)
save_path = '/Users/rince/Desktop/Horse/Data/saishin/results.csv'
results = load_csv(save_path)
save_path = '/Users/rince/Desktop/Horse/Data/saishin/peds.csv'
peds = load_csv(save_path)
save_path = '/Users/rince/Desktop/Horse/Data/saishin/horse_results.csv'
horse_results = load_csv(save_path)

In [6]:
horse_results.columns

Index(['日付', '開催', '天気', 'R', 'レース名', '映像', '頭数', '枠番', '馬番', 'オッズ', '人気',
       '着順', '騎手', '斤量', '距離', '馬場', '馬場指数', 'タイム', '着差', 'ﾀｲﾑ指数', '通過', 'ペース',
       '上り', '馬体重', '厩舎ｺﾒﾝﾄ', '備考', '勝ち馬(2着馬)', '賞金'],
      dtype='object')

In [8]:
horse_results.index

Int64Index([2017105800, 2017105800, 2017105800, 2017105800, 2017105800,
            2017105800, 2017105800, 2017105800, 2017105800, 2017105800,
            ...
            2017106185, 2017106185, 2017106185, 2017106185, 2017106185,
            2017106185, 2017106185, 2017106185, 2017106185, 2017106185],
           dtype='int64', length=243226)

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,course_len,weather,race_type,ground_state,date,horse_id,jockey_id
202005020906,1,4,8,キタノヴィジョン,牡3,56.0,三浦皇成,2:11.9,NaN,3.8,1.0,468(+2),[東] 萱野浩二,2100,曇,ダート,稍重,2020年5月23日,2017102011,1122
202005020906,2,1,1,ジャッジ,牡3,56.0,Ｍ．デム,2:12.0,3/4,7.3,4.0,500(+16),[東] 和田雄二,2100,曇,ダート,稍重,2020年5月23日,2017102704,5212
202005020906,3,6,12,アメリカンフェイス,牡3,56.0,吉田豊,2:12.0,ハナ,8.8,5.0,472(0),[東] 戸田博文,2100,曇,ダート,稍重,2020年5月23日,2017110066,733
202005020906,4,7,13,ミュアウッズ,牝3,54.0,レーン,2:12.6,3.1/2,4.2,2.0,580(-10),[東] 尾関知人,2100,曇,ダート,稍重,2020年5月23日,2017105276,5585
202005020906,5,5,10,ヤマノマタカ,牡3,56.0,石川裕紀,2:13.1,3,22.9,8.0,466(-2),[東] 相沢郁,2100,曇,ダート,稍重,2020年5月23日,2017103008,1150


In [16]:
return_tables.head()

,0,1,2,3
race_id,,,,
202001010101,単勝,6,1600,3
202001010101,複勝,6br2,490br130,4br2
202001010101,馬連,2 - 6,1410,4
202001010101,ワイド,2 - 6br3 - 6br2 - 3,150br170br110,3br4br1
202001010101,馬単,6 → 2,3920,9


In [17]:
# peds.index.name = 'horse_id'
peds.head()

,peds_0,peds_1,peds_2,peds_3,peds_4,peds_5,peds_6,peds_7,peds_8,peds_9,...,peds_52,peds_53,peds_54,peds_55,peds_56,peds_57,peds_58,peds_59,peds_60,peds_61
horse_id,,,,,,,,,,,,,,,,,,,,,
2017100590,クロフネ 1998 芦毛 [血統][産駒] Deputy Minister系,ハッピーティア 2008 鹿毛 [血統][産駒] FNo.[1-w],フレンチデピュティ French Deputy(米) 1992 栗毛 [血統][産駒],ブルーアヴェニュー Blue Avenue(米) 1990 芦毛 [血統][産駒],ホワイトマズル White Muzzle(英) 1990 鹿毛 [血統][産駒],ハッピーチケット 2004 栗毛 [血統][産駒],Deputy Minister 1979 黒鹿毛 [血統][産駒],Mitterand 1981 鹿毛 [血統][産駒],Classic Go Go 1978 黒鹿毛 [血統][産駒],Eliza Blue 1983 芦毛 [血統][産駒],...,Tyrant,Flight Table,Halo,Wishing Well,Lyphard,Prodice,Herbager,Silver Coin,チャイナロック,ソシアルバターフライ
2017106441,スピルバーグ 2009 鹿毛 [血統][産駒] Halo系,ミュージー Musee(米) 2001 芦毛 [血統][産駒] FNo.[4-r],ディープインパクト 2002 鹿毛 [血統][産駒],プリンセスオリビア Princess Olivia(米) 1995 栗毛 [血統][産駒],El Prado 1989 芦毛 [血統][産駒],Foible 1994 栗毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],ウインドインハーヘア Wind in Her Hair(愛) 1991 鹿毛 [血統][産駒],Lycius 1988 栗毛 [血統][産駒],Dance Image 1990 鹿毛 [血統][産駒],...,Tom Fool,Ghazni,Nasrullah,Lalun,Prince John,Nile Lily,Turn-to,Your Game,Restless Native,Enchanted Eve
2017106225,ケープブランコ Cape Blanco() 2007 [血統][産駒] Sadler's W...,ヒラボクキャロル 2004 黒鹿毛 [血統][産駒] FNo.[9-f],Galileo (愛) 1998 鹿毛 [血統][産駒],Laurel Delight 1990 栗毛 [血統][産駒],ダンスインザダーク 1993 鹿毛 [血統][産駒],チョウカイキャロル 1991 栗毛 [血統][産駒],Sadler's Wells (米) 1981 鹿毛 [血統][産駒],アーバンシー Urban Sea(米) 1989 栗毛 [血統][産駒],Presidium 1982 鹿毛 [血統][産駒],Foudroyer 1980 栗毛 [血統][産駒],...,Key to the Mint,Native Partner,Hail to Reason,Bramalea,Graustark,Golden Trail,Raise a Native,Gold Digger,Vaguely Noble,Sleek Dancer
2014105693,キンシャサノキセキ 2003 鹿毛 [血統][産駒] Halo系,マイワイルドローズ My Wild Rose(米) 1995 鹿毛 [血統][産駒] FNo...,フジキセキ 1992 青鹿毛 [血統][産駒],ケルトシャーン Keltshaan(米) 1994 鹿毛 [血統][産駒],Wild Again 1980 黒鹿毛 [血統][産駒],Rose of Mull 1988 栗毛 [血統][産駒],サンデーサイレンス Sunday Silence(米) 1986 青鹿毛 [血統][産駒],ミルレーサー Millracer(米) 1983 鹿毛 [血統][産駒],Pleasant Colony 1978 黒鹿毛 [血統][産駒],Featherhill 1978 鹿毛 [血統][産駒],...,Dante,Clovelly,Sword Dancer,Kerala,Buckpasser,Intriguing,Ribot,Pocahontas,Dark Star,Vulcania
2013101632,メイショウサムソン 2003 鹿毛 [血統][産駒] Sadler's Wells系,シルクヴァルキリー 2003 鹿毛 [血統][産駒] FNo.[7-f],オペラハウス Opera House(英) 1988 鹿毛 [血統][産駒],マイヴィヴィアン 1997 鹿毛 [血統][産駒],マーベラスサンデー 1992 栃栗毛 [血統][産駒],シルクフレアー 1995 鹿毛 [血統][産駒],Sadler's Wells (米) 1981 鹿毛 [血統][産駒],Colorspin 1983 鹿毛 [血統][産駒],ダンシングブレーヴ Dancing Brave(米) 1983 鹿毛 [血統][産駒],ウイルプリンセス 1983 鹿毛 [血統][産駒],...,Laugh Aloud,Hold Me Close,Nearctic,Natalma,Admiral's Voyage,Petitioner,Raise a Native,Sweet Tooth,ノノアルコ,Mortefontaine


In [18]:
# horse_results.index.name = 'horse_id'
horse_results.head()

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
horse_id,,,,,,,,,,,,,,,,,,,,,
2017105800,2021/07/08,名古屋,曇,7.0,C12組,NaN,10.0,4.0,4,6.6,...,NaN,**,4-3,0.0-39.5,NaN,479(-1),NaN,NaN,ナツミフルーリー,NaN
2017105800,2021/06/30,名古屋,曇,5.0,C14組,NaN,10.0,3.0,3,12.2,...,0.7,**,5-6-5,0.0-40.3,39.4,480(-4),NaN,NaN,マルカブレーブ,10.2
2017105800,2021/06/16,名古屋,雨,5.0,美穂ちゃんバースデイ,NaN,11.0,2.0,2,13.5,...,2.7,**,10-10-8,0.0-38.2,38.4,484(+3),NaN,NaN,タイキスピネル,2.6
2017105800,2021/06/03,名古屋,晴,6.0,C14組,NaN,10.0,8.0,9,3.8,...,1.0,**,3-4-4,0.0-39.6,40.1,481(-6),NaN,NaN,トーホウボッカ,NaN
2017105800,2021/05/19,名古屋,雨,5.0,C15組,NaN,11.0,8.0,11,29.8,...,0.5,**,3-3-3,0.0-38.4,38.6,487(0),NaN,NaN,キングリアリティー,10.2


In [10]:
horse_results.head()

,日付,開催,天気,R,レース名,映像,頭数,枠番,馬番,オッズ,...,着差,ﾀｲﾑ指数,通過,ペース,上り,馬体重,厩舎ｺﾒﾝﾄ,備考,勝ち馬(2着馬),賞金
2018102410,2021/10/08,大井,晴,5.0,C2一 二三,NaN,14.0,8.0,13,44.3,...,2.8,**,1-1-3-4,36.9-41.2,43.4,462(+6),NaN,NaN,ピースフルダンス,NaN
2018102410,2021/09/21,大井,晴,1.0,エキサイティングリレ,NaN,11.0,6.0,6,33.4,...,1.1,**,3-3-4-4,37.3-39.6,40.4,456(+1),NaN,NaN,ラウンシー,16.0
2018102410,2021/09/10,大井,晴,2.0,2019ライトニング,NaN,14.0,5.0,7,58.7,...,2.6,**,4-4,35.1-38.3,40.6,455(-1),NaN,NaN,ヨウコウザクラ,NaN
2018102410,2021/08/20,大井,晴,6.0,3歳 314万,NaN,14.0,3.0,3,16.6,...,2.9,**,12-12,35.8-37.5,39.2,456(+4),NaN,NaN,ラストヴィグラス,NaN
2018102410,2021/02/18,大井,晴,1.0,3歳 340万,NaN,14.0,5.0,8,26.9,...,0.9,**,5-5,36.4-37.8,38.5,452(-9),NaN,NaN,アークリオン,33.0


In [111]:
cursor.execute_many(df_nissan_new,'02Nissan')


UNIQUE constraint failed: Hiashis.id, Hiashis.date


In [114]:
db = cursor.return_db()
sql = """ SELECT MAX(Hiashis.date) FROM Hiashis WHERE Hiashis.meigara_id = 2"""
count = 0
for i in  db.execute(sql):
    date_ = i[0]
    count+=1
    if count>=10:
        break

In [119]:
db.execute(sql).fetchone()[0]

'2021-06-30'

In [115]:
date_

'2021-06-30'

# ユニークindex

In [109]:
sql = """CREATE UNIQUE INDEX date_check
 ON Hiashis(meigara_id,date);"""
cursor.return_db().execute(sql)

# Results table

In [16]:
sql = """
    CREATE TABLE Results (
        id INTEGER ,
        chakujun VARCHAR(40),
        wakuban INTEGER,
        umaban INTEGER,
        umamei VARCHAR(40),
        seirei VARCHAR(40),
        kinryo REAL,
        kishu VARCHAR(40),
        time VARCHAR(40),
        chakusa VARCHAR(40),
        tansho VARCHAR(40),
        ninki REAL,
        umataiju VARCHAR(40),
        chokyoshi VARCHAR(40),
        course_len INTEGER,
        weather VARCHAR(40),
        race_type VARCHAR(40),
        ground_state VARCHAR(40),
        date VARCHAR(40),
        horse_id INTEGER,
        jocky_id INTEGER 
    );
"""
cursor.create_table(sql)

# Return table

In [106]:
sql = """
    CREATE TABLE Hiashis (
        meigara_id INTEGER,
        date VARCHAR(20),
        open REAL,
        high REAL,
        low  REAL,
        close REAL,
        volume REAL
    );
"""
cursor.create_table(sql)

# Peds table

In [165]:
sql = """
    CREATE TABLE Ayumines (
        id INTEGER ,
        date VARCHAR(20),
        time VARCHAR(20),
        price REAL,
        volume REAL,
        total_price  REAL
    );
"""
cursor.create_table(sql)

# Horse_results table

In [ ]:
sql = """
    CREATE TABLE Ayumines (
        id INTEGER ,
        date VARCHAR(20),
        time VARCHAR(20),
        price REAL,
        volume REAL,
        total_price  REAL
    );
"""
cursor.create_table(sql)

# SQLAlchemy

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import scoped_session, sessionmaker
from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
import os

In [2]:
# current working directory
os.getcwd()

'/Users/rince/Desktop/Horse/code/horse'

In [3]:
database_file = os.path.join(os.path.abspath(os.getcwd()),'data.db')

In [10]:
database_file

'/Users/rince/Desktop/Horse/code/horse/data.db'

おまじない

In [4]:
enigine = create_engine('sqlite:///'+database_file,convert_unicode=True, echo=True)

In [6]:
db_session = scoped_session(
    sessionmaker(
        autocommit=False,
        autoflush=False,
        blind = enigine
    )
)

In [7]:
Base = declarative_base()
Base.query = db_session.query_property()

In [8]:

Base.metadata.create_all(bind=enigine)

2022-04-28 18:39:54,978 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2022-04-28 18:39:54,979 INFO sqlalchemy.engine.base.Engine ()
2022-04-28 18:39:54,980 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2022-04-28 18:39:54,982 INFO sqlalchemy.engine.base.Engine ()


In [9]:
from sqlalchemy import Column, Integer, Float

In [ ]:
# テーブル記述
class Horse(Base):
    __tablename__= "horse_class"
    id = Column(Integer,primary_key=True)
    